In [36]:
import sys
sys.path.insert(0, '../')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
import io
import tabulate
import helpers_eda_inicial
import helpers_cbc

In [ ]:
cbc, df_carreras = helpers_cbc.get_data('../../../assets/bronze/CBC/Sitacad_Tesis (1).xlsx')
actas = pd.read_csv('../../../assets/bronze/FCEN/FCEN_oficial_v2/reportes_actas.csv')
personas = pd.read_csv('../../../assets/bronze/FCEN/FCEN_oficial_v3/reporte_personas.csv')

In [ ]:
cbc['Fecha'].unique()

In [39]:
cbc['Dni'] = cbc['Dni'].astype(str)
actas['fecha'] = pd.to_datetime(actas['fecha'], format='%Y-%m-%d')
actas['año'] = actas['fecha'].dt.year
actas['mes'] = actas['fecha'].dt.month

In [ ]:
actas['año'].value_counts()

In [ ]:
# Contar la cantidad de ocurrencias por mes
conteo_meses = actas['mes'].value_counts().sort_index()

# Crear el barplot
plt.figure(figsize=(10, 6))
sns.barplot(x=conteo_meses.index, y=conteo_meses.values, palette='viridis')

plt.xlabel('Mes')
plt.ylabel('Cantidad de actas')
plt.title('Cantidad de actas por mes')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Contar la cantidad de actas por año y mes
conteo = actas[(actas['año'] > 2019) & (actas['año'] < 2025)].groupby(['año', 'mes']).size().reset_index(name='cantidad')

# Calcular el total de actas por año
totales_por_año = conteo.groupby('año')['cantidad'].transform('sum')

# Calcular el porcentaje
conteo['porcentaje'] = conteo['cantidad'] / totales_por_año * 100

# Asegurar el orden correcto de los meses si están en texto
#orden_meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
#               'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
#conteo['mes'] = pd.Categorical(conteo['mes'], categories=orden_meses, ordered=True)

# Crear el barplot agrupado
plt.figure(figsize=(12, 6))
sns.barplot(data=conteo, x='mes', y='porcentaje', hue='año', palette='viridis')

plt.xlabel('Mes', fontsize=15)
plt.ylabel('Porcentaje de actas (%)', fontsize=15)
plt.title('Distribución porcentual de actas por mes y año', fontsize=15)
plt.yticks(fontsize=13)
plt.xticks(rotation=0, fontsize=13)
plt.legend(title='Año', fontsize=13)
plt.tight_layout()
plt.show()

In [ ]:
# Contar la cantidad de actas por año y mes
conteo = actas[(actas['tipo_acta'] == 'Acta de Regulares/Promociones') & 
                (actas['año'] > 2019) & (actas['año'] < 2025)].groupby(['año', 'mes']).size().reset_index(name='cantidad')

# Calcular el total de actas por año
totales_por_año = conteo.groupby('año')['cantidad'].transform('sum')

# Calcular el porcentaje
conteo['porcentaje'] = conteo['cantidad'] / totales_por_año * 100

# Asegurar el orden correcto de los meses si están en texto
#orden_meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
#               'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
#conteo['mes'] = pd.Categorical(conteo['mes'], categories=orden_meses, ordered=True)

# Crear el barplot agrupado
plt.figure(figsize=(12, 6))
sns.barplot(data=conteo, x='mes', y='porcentaje', hue='año', palette='viridis')

plt.xlabel('Mes')
plt.ylabel('Porcentaje de actas (%)')
plt.title('Distribución porcentual de actas por mes y año de TPs')
plt.xticks(rotation=45)
plt.legend(title='Año')
plt.tight_layout()
plt.show()


In [ ]:
# Contar la cantidad de actas por año y mes
conteo = actas[(actas['tipo_acta'] == 'Acta de Examen') & 
                (actas['año'] > 2019) & (actas['año'] < 2025)].groupby(['año', 'mes']).size().reset_index(name='cantidad')

# Calcular el total de actas por año
totales_por_año = conteo.groupby('año')['cantidad'].transform('sum')

# Calcular el porcentaje
conteo['porcentaje'] = conteo['cantidad'] / totales_por_año * 100

# Asegurar el orden correcto de los meses si están en texto
#orden_meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
#               'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
#conteo['mes'] = pd.Categorical(conteo['mes'], categories=orden_meses, ordered=True)

# Crear el barplot agrupado
plt.figure(figsize=(12, 6))
sns.barplot(data=conteo, x='mes', y='porcentaje', hue='año', palette='viridis')

plt.xlabel('Mes')
plt.ylabel('Porcentaje de actas (%)')
plt.title('Distribución porcentual de actas por mes y año de Finales')
plt.xticks(rotation=45)
plt.legend(title='Año')
plt.tight_layout()
plt.show()


In [45]:
actas_fechas_raras = actas[~actas['mes'].isin([7,8,12])]

In [ ]:
actas_fechas_raras['tipo_acta'].value_counts()

In [ ]:
actas_fechas_raras['resultado'].value_counts()

In [48]:
actas['semestre'] = np.where(actas['mes'] <= 8, 1, 2)
#actas['semestre'] = actas['mes'].apply(lambda x: 1 if x in [12, 1, 2, 3, 4, 5] else 2)
actas['periodo_int'] = actas['año'] * 10 + actas['semestre']

Tomandolo como puse arriba queda re desbalanceado

In [ ]:
actas['semestre'].value_counts()

In [50]:
en_cbc = cbc['Dni'].unique()
actas_con_cbc = actas[actas['dni'].isin(en_cbc)]
personas_con_cbc = personas[personas['dni'].isin(en_cbc)]

Ahora no hay actas anteriores a 2020

In [ ]:
actas_con_cbc['fecha'].min()

In [ ]:
personas['año_inscripcion_facultad'].value_counts(dropna=False).sort_index().plot(kind='bar', figsize=(10, 5), title='Año de inscripción en la facultad sin registro de CBC')

In [ ]:
personas_con_cbc['año_inscripcion_facultad'].value_counts(dropna=False).sort_index().plot(kind='bar', figsize=(10, 5), title='Año de inscripción en la facultad con registro de CBC')

Acá no filtré por año

In [54]:
# Ordenamos y asignamos el cuatrimestre relativo (0, 1, 2, ...) por alumno
actas_con_cbc = actas_con_cbc.sort_values(['dni', 'periodo_int'])
actas_con_cbc['cuatrimestre_relativo'] = actas_con_cbc.groupby('dni')['periodo_int'].rank(method='dense').astype(int) - 1

# Paso 2: Filtrar solo aprobados
actas_aprobados = actas_con_cbc[actas_con_cbc['resultado'] == 'Aprobado']

# Paso 3: Contar aprobadas por alumno y cuatrimestre relativo
aprobados_por_alumno = actas_aprobados.groupby(['dni', 'cuatrimestre_relativo']).size().reset_index(name='cantidad_aprobadas')

In [ ]:
frecuencia = aprobados_por_alumno.groupby(['cuatrimestre_relativo', 'cantidad_aprobadas']) \
    .size().unstack(fill_value=0)

plt.figure(figsize=(12, 6))
sns.heatmap(frecuencia.T, cmap='YlGnBu', annot=True, fmt='d', cbar_kws={'label': 'Cantidad de Alumnos'})

plt.title('Distribución de Materias Aprobadas por Cuatrimestre Relativo')
plt.xlabel('Cuatrimestre Relativo')
plt.ylabel('Materias Aprobadas')
plt.tight_layout()
plt.show()

me fijo si en actas_aprobados aparece toda la gente que teníamos originalmente

In [ ]:
actas_aprobados['dni'].nunique(), personas_con_cbc['dni'].nunique(), actas_con_cbc['dni'].nunique(), cbc['Dni'].nunique()

Veo que en actas_aprobados y, en realidad, ya en actas_con_cbc, hay varios alumnos que no aparecen. Igualmente, no podríamos usarlos para el modelo porque necesitamos tener aunque sea un cuatrimestre. Sin embargo, puede ser gente que comenzó su cursada en 2025

Agrego el año de inscripción

In [57]:
anio_inscripcion = personas[['dni', 'año_inscripcion_facultad']]
actas_cbc_con_anio_inscripcion = pd.merge(actas_con_cbc, anio_inscripcion, left_on='dni', right_on='dni', how='left')

In [58]:
def heatmap_aprobadas(actas, anio):
    # Ordenamos y asignamos el cuatrimestre relativo (0, 1, 2, ...) por alumno
    actas = actas.sort_values(['dni', 'periodo_int'])
    actas['cuatrimestre_relativo'] = actas.groupby('dni')['periodo_int'].rank(method='dense').astype(int) - 1

    # Paso 2: Filtrar solo aprobados
    actas_aprobados = actas[(actas['resultado'] == 'Aprobado') & (actas['año_inscripcion_facultad'] == anio)]

    # Paso 3: Contar aprobadas por alumno y cuatrimestre relativo
    aprobados_por_alumno = actas_aprobados.groupby(['dni', 'cuatrimestre_relativo']).size().reset_index(name='cantidad_aprobadas')

    frecuencia = aprobados_por_alumno.groupby(['cuatrimestre_relativo', 'cantidad_aprobadas']) \
        .size().unstack(fill_value=0)

    plt.figure(figsize=(12, 6))
    sns.heatmap(frecuencia.T, cmap='YlGnBu', annot=True, fmt='d', cbar_kws={'label': 'Cantidad de Alumnos'})

    plt.title(f'Distribución de Materias Aprobadas por Cuatrimestre Relativo. Año de Inscripción: {anio}')
    plt.xlabel('Cuatrimestre Relativo')
    plt.ylabel('Materias Aprobadas')
    plt.tight_layout()
    plt.show()

In [ ]:
for anio in range(2020, 2025):
    heatmap_aprobadas(actas_cbc_con_anio_inscripcion, anio)

Quiero poder ver si aprobaron 0 materias. Vuelvo a ver primero sin prestar atención al CBC

In [60]:
# Paso 2.5: Construir todas las combinaciones posibles (incluso si no aprobó nada)
actas = actas.sort_values(['dni', 'periodo_int'])
actas['cuatrimestre_relativo'] = actas.groupby('dni')['periodo_int'].rank(method='dense').astype(int) - 1
actas_con_anio_inscripcion = pd.merge(actas, anio_inscripcion, left_on='dni', right_on='dni', how='left')

In [ ]:
# Paso 1: Filtrar aprobados
df_aprobados = actas_con_anio_inscripcion[actas_con_anio_inscripcion['resultado'] == 'Aprobado']

# Paso 2 y 3: Agrupar por dni, año_inscripcion_facultad y cuatrimestre_relativo
conteo = df_aprobados.groupby(['dni', 'año_inscripcion_facultad', 'cuatrimestre_relativo']) \
                     .size().reset_index(name='cantidad_aprobadas')

# Paso 4: Pivotear
tabla_pivot = conteo.pivot(index=['dni', 'año_inscripcion_facultad'], 
                           columns='cuatrimestre_relativo', 
                           values='cantidad_aprobadas')

# Renombrar columnas
tabla_pivot.columns = [f'cuatrimestre_{i}' for i in tabla_pivot.columns]

# Reset index para que dni y año_inscripcion_facultad estén como columnas
tabla_final = tabla_pivot.reset_index()

tabla_final.head()

In [ ]:
df_melted = tabla_final.melt(id_vars=['dni', 'año_inscripcion_facultad'], 
                             var_name='cuatrimestre', 
                             value_name='cantidad_aprobadas')

# Extraemos el número de cuatrimestre desde el string (ej: 'cuatrimestre_0' -> 0)
df_melted['cuatrimestre'] = df_melted['cuatrimestre'].str.extract('(\d+)').astype(int)

# Ahora agrupamos por cuatrimestre y cantidad_aprobadas para contar alumnos
heatmap_data = df_melted.groupby(['cuatrimestre', 'cantidad_aprobadas']) \
                        .size().unstack(fill_value=0)

# Hacemos el heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(heatmap_data.T, annot=True, fmt='d', cmap='YlGnBu')
plt.xlabel('Cuatrimestre relativo')
plt.ylabel('Cantidad de materias aprobadas')
plt.title('Cantidad de alumnos según cuatrimestre y materias aprobadas')
plt.tight_layout()
plt.show()


Vamos de nuevo pero tomando solo la gente con registro del CBC

In [ ]:
actas_cbc_con_anio_inscripcion.head()

In [ ]:
# Paso 1: Filtrar aprobados
df_aprobados = actas_cbc_con_anio_inscripcion[actas_cbc_con_anio_inscripcion['resultado'] == 'Aprobado']

# Paso 2 y 3: Agrupar por dni, año_inscripcion_facultad y cuatrimestre_relativo
conteo = df_aprobados.groupby(['dni', 'año_inscripcion_facultad', 'cuatrimestre_relativo']) \
                     .size().reset_index(name='cantidad_aprobadas')

# Paso 4: Pivotear
tabla_pivot = conteo.pivot(index=['dni', 'año_inscripcion_facultad'], 
                           columns='cuatrimestre_relativo', 
                           values='cantidad_aprobadas')

# Rellenar NaN con 0 y convertir a int
tabla_pivot = tabla_pivot.fillna(0).astype(int)

# Renombrar columnas
tabla_pivot.columns = [f'cuatrimestre_{i}' for i in tabla_pivot.columns]

# Reset index para que dni y año_inscripcion_facultad estén como columnas
tabla_final = tabla_pivot.reset_index()

tabla_final.head()

In [ ]:
df_melted = tabla_final.melt(id_vars=['dni', 'año_inscripcion_facultad'], 
                             var_name='cuatrimestre', 
                             value_name='cantidad_aprobadas')

# Extraemos el número de cuatrimestre desde el string (ej: 'cuatrimestre_0' -> 0)
df_melted['cuatrimestre'] = df_melted['cuatrimestre'].str.extract('(\d+)').astype(int)

# Ahora agrupamos por cuatrimestre y cantidad_aprobadas para contar alumnos
heatmap_data = df_melted.groupby(['cuatrimestre', 'cantidad_aprobadas']) \
                        .size().unstack(fill_value=0)

# Hacemos el heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(heatmap_data.T, annot=True, fmt='d', cmap='YlGnBu')
plt.xlabel('Cuatrimestre relativo')
plt.ylabel('Cantidad de materias aprobadas')
plt.title('Cantidad de alumnos según cuatrimestre y materias aprobadas sin filtro año de inscripción')
plt.tight_layout()
plt.show()

In [66]:
def heatmap_con_filtro_anio(anio):
    # Paso 1: Filtrar aprobados
    df_aprobados = actas_cbc_con_anio_inscripcion[(actas_cbc_con_anio_inscripcion['resultado'] == 'Aprobado')
                                                   & (actas_cbc_con_anio_inscripcion['año_inscripcion_facultad'] == anio)]

    # Paso 2 y 3: Agrupar por dni, año_inscripcion_facultad y cuatrimestre_relativo
    conteo = df_aprobados.groupby(['dni', 'año_inscripcion_facultad', 'cuatrimestre_relativo']) \
                        .size().reset_index(name='cantidad_aprobadas')

    # Paso 4: Pivotear
    tabla_pivot = conteo.pivot(index=['dni', 'año_inscripcion_facultad'], 
                            columns='cuatrimestre_relativo', 
                            values='cantidad_aprobadas')

    # Rellenar NaN con 0 y convertir a int
    tabla_pivot = tabla_pivot.fillna(0).astype(int)

    # Renombrar columnas
    tabla_pivot.columns = [f'cuatrimestre_{i}' for i in tabla_pivot.columns]

    # Reset index para que dni y año_inscripcion_facultad estén como columnas
    tabla_final = tabla_pivot.reset_index()

    df_melted = tabla_final.melt(id_vars=['dni', 'año_inscripcion_facultad'], 
                             var_name='cuatrimestre', 
                             value_name='cantidad_aprobadas')

    # Extraemos el número de cuatrimestre desde el string (ej: 'cuatrimestre_0' -> 0)
    df_melted['cuatrimestre'] = df_melted['cuatrimestre'].str.extract('(\d+)').astype(int)

    # Ahora agrupamos por cuatrimestre y cantidad_aprobadas para contar alumnos
    heatmap_data = df_melted.groupby(['cuatrimestre', 'cantidad_aprobadas']) \
                            .size().unstack(fill_value=0)

    # Hacemos el heatmap
    plt.figure(figsize=(12, 6))
    sns.heatmap(heatmap_data.T, annot=True, fmt='d', cmap='YlGnBu')
    plt.xlabel('Cuatrimestre relativo')
    plt.ylabel('Cantidad de materias aprobadas')
    plt.title('Cantidad de alumnos según cuatrimestre y materias aprobadas inscriptos en ' + str(anio))
    plt.tight_layout()
    plt.show()

In [ ]:
for anio in range(2020, 2025):
    heatmap_con_filtro_anio(anio)

In [68]:
def heatmap_con_filtro_anio_tipo_acta(anio):
    # Paso 1: Filtrar aprobados
    df_aprobados = actas_cbc_con_anio_inscripcion[(actas_cbc_con_anio_inscripcion['resultado'] == 'Aprobado')
                                                    & (actas_cbc_con_anio_inscripcion['tipo_acta'] == 'Acta de Examen')
                                                   & (actas_cbc_con_anio_inscripcion['año_inscripcion_facultad'] == anio)]

    # Paso 2 y 3: Agrupar por dni, año_inscripcion_facultad y cuatrimestre_relativo
    conteo = df_aprobados.groupby(['dni', 'año_inscripcion_facultad', 'cuatrimestre_relativo']) \
                        .size().reset_index(name='cantidad_aprobadas')

    # Paso 4: Pivotear
    tabla_pivot = conteo.pivot(index=['dni', 'año_inscripcion_facultad'], 
                            columns='cuatrimestre_relativo', 
                            values='cantidad_aprobadas')

    # Rellenar NaN con 0 y convertir a int
    tabla_pivot = tabla_pivot.fillna(0).astype(int)

    # Renombrar columnas
    tabla_pivot.columns = [f'cuatrimestre_{i}' for i in tabla_pivot.columns]

    # Reset index para que dni y año_inscripcion_facultad estén como columnas
    tabla_final = tabla_pivot.reset_index()

    df_melted = tabla_final.melt(id_vars=['dni', 'año_inscripcion_facultad'], 
                             var_name='cuatrimestre', 
                             value_name='cantidad_aprobadas')

    # Extraemos el número de cuatrimestre desde el string (ej: 'cuatrimestre_0' -> 0)
    df_melted['cuatrimestre'] = df_melted['cuatrimestre'].str.extract('(\d+)').astype(int)

    # Ahora agrupamos por cuatrimestre y cantidad_aprobadas para contar alumnos
    heatmap_data = df_melted.groupby(['cuatrimestre', 'cantidad_aprobadas']) \
                            .size().unstack(fill_value=0)

    # Hacemos el heatmap
    plt.figure(figsize=(12, 6))
    sns.heatmap(heatmap_data.T, annot=True, fmt='d', cmap='YlGnBu')
    plt.xlabel('Cuatrimestre relativo')
    plt.ylabel('Cantidad de materias aprobadas')
    plt.title('Cantidad de alumnos según cuatrimestre y materias aprobadas inscriptos en ' + str(anio))
    plt.tight_layout()
    plt.show()

In [ ]:
for anio in range(2020, 2025):
    heatmap_con_filtro_anio_tipo_acta(anio)